# 04-extract_tdms

This notebook extracts time delayed melody surface from the predominant melody of each audio recording. The feature was introduced in:

> Gulati S, Serrà J, Ganguli K K, Şentürk S, Serra X. Time-delayed melody surfaces for Rāga recognition. In Proceedings of the 17th International Society for Music Information Retrieval Conference (ISMIR 2016); 2016 Aug 7-11; New York City (NY). p. 751-7.


In [1]:
import mre


## Stop if pitch class distibution was run in the past

Extracting the feature may take a long time. It is better to check if there is already a run in advance.

In [4]:
run = mre.mlflow_common.get_run_by_name(
    mre.data.TimeDelayedMelodySurface.EXPERIMENT_NAME,
    mre.data.TimeDelayedMelodySurface.RUN_NAME)

if run is not None:
    raise ValueError(
        "There is already a run for %s:%s. Overwriting is not "
        "permitted. Please delete the run manually if you want "
        "to extract the time delayed melody surfaces again."
        % (mre.data.TimeDelayedMelodySurface.RUN_NAME, run.run_id))


# Read predominant melody filepaths from mlflow


In [5]:
melody_paths = mre.data.PredominantMelodyMakam.from_mlflow()
display(melody_paths[:5])

['/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/006536f8-bf54-4cc0-a510-5a52456d09f8.npy',
 '/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/009309d2-c260-4808-8f1d-44a5ddc6bc5f.npy',
 '/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/00a48b5f-a35a-436c-a7a0-4438130f4abf.npy',
 '/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/00ab81ec-07f8-47ba-9610-47ad56393eb9.npy',
 '/data/artifacts/1/2608c404325846309549b68e5020bd17/artifacts/00be9a4b-b85f-4601-a6d5-9ce1f5b3f91c.npy']

# Read tonic annotations from mlflow

In [6]:
annotations = mre.data.Annotation.from_mlflow()
tonic_frequencies = annotations.data.set_index("mbid")["tonic"]
tonic_frequencies.head()


mbid
00f1c6d9-c8ee-45e3-a06f-0882ebcb4e2f    256.0
168f7c75-84fb-4316-99d7-acabadd3b2e6    115.2
24f549dd-3fa4-4e9b-a356-778fbbfd5cad    232.5
407bb0b4-f19b-42ab-8c0a-9f1263126951    233.5
443819eb-6092-420c-bd86-d946a0ad6555    219.6
Name: tonic, dtype: float64

# Extract TDMSs from normalized predominant melody features and log to mlflow


In [9]:
tdms = mre.data.TimeDelayedMelodySurface()
tdms.transform(melody_paths, tonic_frequencies)
tdms.log()


100%|██████████| 1000/1000 [02:46<00:00,  5.99it/s]
No runs with the name time_delayed_melody_surface in experiment data_processing
